In [20]:
import datasets
ds = datasets.load_from_disk("./taco_cleaned")
ds

Dataset({
    features: ['question', 'solutions', 'starter_code', 'input_output', 'difficulty', 'raw_tags', 'name', 'source', 'tags', 'skill_types', 'url', 'Expected Auxiliary Space', 'time_limit', 'date', 'picture_num', 'memory_limit', 'Expected Time Complexity'],
    num_rows: 9030
})

In [21]:
# figure out format of tests
import json
import sys
sys.set_int_max_str_digits(0)
keys = set()
types = set()
has_fn_name = 0

for ex in ds:
    eio = ex["input_output"]
    eio = json.loads(eio)
    types.add(type(eio))
    ks = eio.keys()
    if "fn_name" in ks:
        has_fn_name += 1
    for k in ks:
        keys.add(k)

print(types)
print(keys)
print(has_fn_name)

{<class 'dict'>}
{'inputs', 'fn_name', 'outputs'}
544


In [36]:
9000 * 5

# MinHash + Locality Sentitive Hasking, Jaccard Similarity 0.5
# pick 5 dedup
# 1 with original prompt, 4 LLM to rephrase the prompt

45000

In [15]:
import json
print(json.loads(codewars[1000]["solutions"])[0])

def palindrome_pairs(w):
	return [[i, j] for i in range(len(w)) for j in range(len(w)) if str(w[i]) + str(w[j]) == (str(w[i]) + str(w[j]))[::-1] and i != j]

